### Capstone Project - Segmentation and Clustering of Toronto Canada
#### Step one 
- Import the Library required

In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json # library to handle JSON files
import requests # library to handle requests
import csv
import sys
trace = False

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # import k-means from clustering stage
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Step two
- Scrap the data from Wikipedia

In [24]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(wiki_url,'lxml')
#print(soup.prettify())

df = soup.find('table',{'class':'wikitable sortable'})
#print(df)


#### Step three
- Get the features from Wikipedia table

In [25]:
feature_names = []

header_row = df.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))
    feature_name = feature_name.replace('\n', '')
    feature_names.append(feature_name)

print(feature_names)

['Postcode', 'Borough', 'Neighbourhood']


- Extract the table

In [26]:
samples = []
sample_rows = df.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.string
        if text:
            if trace:
                features.append('T = {}'.format(text))
            else:
                features.append(text)
            continue
        
        for child in feature_col.children:
            if child.name == 'span':
                if child.has_attr('class'):
                    if child['class'] == 'display:none':
                        continue
                if child.find_all(has_coords):
                    feature_value = get_coords(child)
                    if feature_value:
                        break
                    else:
                        continue
            if child.name == 'sup':
                continue
            if child.name == 'a':
                if child.string[0] == '[':
                    continue            
            if child.name == 'a':
                if trace:
                    feature_value = 'A = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            if child.name == 'font':
                if trace:
                    feature_value = 'F = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            try:
                # feature_value = '' for any tags not covered above
                content = child.contents
            except AttributeError:
                # Handle whitespace between child tags, treated as a child string
                if child.isspace():
                    continue
                if trace:
                    feature_value = 'E = {}'.format(child)
                else:
                    feature_value = child
                break
        features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))

data = pd.DataFrame(samples)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()),
        data.shape[0]
    )
)

The dataframe has 12 boroughs and 288 neighborhoods.


#### Data Cleansing
- Drop the Not assigned Borough
- assign Borough to Not assigned neighbourhood 

In [27]:
data = data.drop(data[data.Borough =='Not assigned'].index)
#data.Neighbourhood = np.where(data.Neighbourhood == 'Not assigned', data.Borough, data.Neighbourhood)
data.Neighbourhood = data['Neighbourhood'].str.replace('\n', '')
data.Borough = data['Borough'].str.replace('\n', '')
data.Neighbourhood[data.Neighbourhood == 'Not assigned'] = data.Borough
print(data[data.Postcode=='M7A'])

        Borough Neighbourhood Postcode
8  Queen's Park  Queen's Park      M7A


#### Change the dimension for the same Borough by creating list of Neighbourhood

In [28]:
p_table = data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
p_table = p_table.reset_index() 
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(p_table['Borough'].unique()),
        p_table.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [29]:
p_table.shape

(103, 3)

In [30]:
p_table['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [31]:
p_table.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [32]:
lat_long_data = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [35]:
p_table = pd.merge(p_table,lat_long_data, left_on='Postcode', right_on='Postal Code', how='left')

In [37]:
p_table_lat_long = p_table.drop('Postal Code', axis = 1)

In [40]:
p_table_lat_long.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
